# See Results

In [1]:
from Helpers import load_tokenizer

tk_body = load_tokenizer('tk_body.dpkl')
tk_title = load_tokenizer('tk_title.dpkl')

/home/richard/deep_learn_p3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Size of vocabulary for tk_body.dpkl: 8000
Size of vocabulary for tk_title.dpkl: 4500


In [2]:
from keras.models import load_model

# Current best 'seq2seq_model_bi1.h5
seq2seq_Model = load_model('seq2seq_model.h5')

/home/richard/deep_learn_p3/lib/python3.5/site-packages/keras/engine/topology.py:1253: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


#### Extract encoder; Re-structure decoder

In [3]:
encoder_model = seq2seq_Model.get_layer('Encoder-Model')

In [4]:
from keras.layers import Input, merge
from keras.models import Model

def extract_decoder_model(model):
    latent_dim = model.get_layer('Title-Word-Embedding').output_shape[-1]
    
    decoder_inputs = model.get_layer('Decoder-Input').input
    dec_emb = model.get_layer('Title-Word-Embedding')(decoder_inputs)
    dec_bn = model.get_layer('Decoder-BatchNorm-1')(dec_emb)
    
    gru_inference_state_input = Input(shape=(latent_dim,), name='hidden_state_input')
    
    gru_out, gru_state_out = model.get_layer('Decoder-GRU')([dec_bn, gru_inference_state_input]) 
#     gru_out_back, gru_state_out_back = model.get_layer('Decoder-Backward-GRU')([dec_bn, gru_inference_state_input]) 
    
#     gru_out = merge([gru_out, gru_out_back], mode='concat')
    
    dec_bn2 = model.get_layer('Decoder-BatchNorm-2')(gru_out)
    
    dense_out = model.get_layer('Final-Output-Dense')(dec_bn2)
    
    decoder_model = Model([decoder_inputs, gru_inference_state_input],
                          [dense_out, gru_state_out])
    return decoder_model

decoder_model = extract_decoder_model(seq2seq_Model)

In [5]:
from Helpers import create_title

body_text = """
I am trying to implement a LSTM based speech recognizer. So far I could set up bidirectional LSTM (i think it is working as a bidirectional LSTM) by following the example in Merge layer. Now I want to try it with another bidirectional LSTM layer, which make it a deep bidirectional LSTM. But I am unable to figure out how to connect the output of the previously merged two layers into a second set of LSTM layers. I don't know whether it is possible with Keras. Hope someone can help me with this.
"""

create_title(body_text, tk_body, tk_title, encoder_model, decoder_model)

Issue Body:
 
I am trying to implement a LSTM based speech recognizer. So far I could set up bidirectional LSTM (i think it is working as a bidirectional LSTM) by following the example in Merge layer. Now I want to try it with another bidirectional LSTM layer, which make it a deep bidirectional LSTM. But I am unable to figure out how to connect the output of the previously merged two layers into a second set of LSTM layers. I don't know whether it is possible with Keras. Hope someone can help me with this.
 


>>>>> Generated Title (Prediction): <<<<<
 how to train the model




#### See some github issues examples

In [6]:
import pandas as pd

df = pd.read_csv('github_issues.csv').sample(n=1000)

In [7]:
body_text = df.body.tolist()
title_text = df.issue_title.tolist()
url = df.issue_url.tolist()

In [9]:
import numpy as np

samples_idx = np.random.randint(0, len(body_text), 5)
for i in samples_idx:
    create_title(body_text[i], tk_body, tk_title, encoder_model, decoder_model, title_text[i])

Issue Body:
 could something like this be possible? <script type= text/uniter > echo this is a php statement ; </script> cheers 

Original Title:
 html tags for uniter

>>>>> Generated Title (Prediction): <<<<<
 add a way to import a file


Issue Body:
 i'm submitting a ... x bug report => search github for a similar issue or pr before submitting feature request => please check if request is not on the roadmap already https://github.com/primefaces/primeng/wiki/roadmap support request => please do not submit support request here, instead see http://forum.primefaces.org/viewforum.php?f=35 current behavior on checkbox mode, when passing selection with list of items, the partial selection behavior doesn't work no fa-minus icon , the partial selected nodes stays empty. it looks like only on click you trigger the propagate up which sets the partialselected to true. i think you should do the same when setting the selection input. expected behavior set partial selected items with fa-minus icon